<h1 align="center">AwakeRodent, RABIES Analysis Code</h1>

<p align="center">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<i>Marie E Galteau, 2023</i></p>


  <table align="center">
    <tr>
      <td align="center"><img src="https://hpc.nih.gov/images/Singularity.png" alt="image" width="40" /></td>
      <td align="center"><img src="https://surfer.nmr.mgh.harvard.edu/pub/data/tmp/brain.png" alt="image" width="150" /></td>
      <td align="center"><img src="https://fsl.fmrib.ox.ac.uk/fsl/wiki_static/fsl/img/fsl-logo-x2.png" alt="image" width="70" /></td>
    </tr>
  </table>
</div>

In [16]:
# -- Load module for singularity -- 

!module load singularity
!module load freesurfer
!module load fsl

import os 
import subprocess
import pandas as pd
from enum import Enum

from IPython.display import display, Markdown, Latex

# -- !! Init Variables !! -- 
scripts_folder='/home/traaffneu/margal/awake_code/awake/scripts/'
metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv'
#metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/online-ds_metadata_rabies.tsv'


df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]

print('CHECK Metadata Path:', metadata_path)

CHECK Metadata Path: /home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv


#### Define RABIES parameters for analysis

In [27]:
# --- Function: analysis, as a job on HPC ---

def qsub_analysis_rabies(subj_num, specie, template_dir, BIDS_input, preprocess_outputs, confound_correction_outputs, analysis_outputs, Cmd_B_rat_template_path, seed_mask_list):
    
    singularity_path='/opt/singularity/3.10.3/bin/singularity'    
    if not os.path.exists(analysis_outputs):os.makedirs(analysis_outputs)      # Create confound_correction_outputs directory

    # -- Create the full command string --
    analysis_cmd = f"{singularity_path} run " \
                    f"{Cmd_B_rat_template_path} " \
                    f"-B {BIDS_input}:/BIDS_input:ro " \
                    f"-B {preprocess_outputs}:/preprocess_outputs/ " \
                    f"-B {confound_correction_outputs}:/confound_correction_outputs/ " \
                    f"-B {analysis_outputs}:/analysis_outputs/ " \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc analysis /confound_correction_outputs/ /analysis_outputs/ " \
                    f"--seed_list {' '.join(seed_mask_list)}" \
                   
                                                     
#f"--scan_QC_thresholds '{Dice:[0.3]}' " \  
                                                     
    # Submit the job
    cmd = f'echo "Job running: $(echo \'{analysis_cmd}\'| qsub -N Awk-A:{specie},{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=48:00:00)"'
    subprocess.call(cmd, shell=True)

#### Define variable + Launch RABIES analysis
... is yet to be functional

In [28]:
# --- Run RABIES ---
    
for index in range(0, 1):
    
    # -- Define variables --
    subj_num=str(df.iloc[index]['rodent.sub'])[:-2]               #remove the 2 last characters with [:-2]
    ses_num=str(df.iloc[index]['rodent.ses'])[:-2] 
    specie = str(df.iloc[index]['rodent.spiecies']) 
    TR=str(df.iloc[index]['func.TR'])[:-2]
    
    analysis_config=str(df.iloc[index]['analysis_config']) 
    analysis_config_list = analysis_config.split(', ') 

    if specie == 'Mouse' :
        specie = 'M'
        bids_folder = '01-03_mice'
        template_dir=''
        Cmd_B_rat_template_path=''
        template=''
        mask=''
        wm=''
        csf=''
        atlas=''
        roi=''
        smoothing = '0.3'
        seed_mask_list=["/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-right_mouse.nii.gz", "/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-left_mouse.nii.gz"]
    
    else : #'Rat'
        specie = 'R'
        bids_folder = '02-04_rats'   
        template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
        Cmd_B_rat_template_path=f"-B {template_dir}:/template"
        template='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
        mask='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii'
        wm='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz'
        csf='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz'
        atlas='/template/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii'
        smoothing='0.5'
        seed_mask_list=["/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-right_rat.nii.gz", "/project/4180000.36/AwakeRodent/scratch/RABIES/template/seed_S1-left_rat.nii.gz"]

    # -- Print scan infos -- 
    scan_infos=(f'Specie: **<span style="color:#FFA07A">{specie}</span>**   \n'
                f'Rodent num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Analysis: <span style="color:#FF91A4">{analysis_config}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n')
    
    display(Markdown(scan_infos)) 

    # -- Input and output directories -- 
    orig_bids_dir=f'/project/4180000.36/AwakeRodent/bids/{bids_folder}/'
    BIDS_input=f'/project/4180000.36/AwakeRodent/scratch/bids/{bids_folder}/sub-0{subj_num}'
    preprocess_outputs=f'/project/4180000.36/AwakeRodent/scratch/RABIES/preprocessed/sub-0{subj_num}_ses-{ses_num}'
    
    confound_correction_outputs_base=preprocess_outputs+'/confound_correction/'
    analysis_outputs_base=preprocess_outputs+'/seed-based_connectivity'

    # -- Launch: Analysis --    
     
    for analysis in analysis_config_list:
        
        regressors_outputs = os.path.join(confound_correction_outputs_base, analysis)
        confound_correction_outputs=regressors_outputs
        
        analysis_outputs=os.path.join(analysis_outputs_base, f"SD_{analysis}")
        #if not os.path.exists(analysis_outputs):os.makedirs(analysis_outputs)  

        qsub_analysis_rabies(subj_num, specie, template_dir, BIDS_input, preprocess_outputs, confound_correction_outputs, analysis_outputs, Cmd_B_rat_template_path, seed_mask_list)    
        print(f"RABIES analysis using: {analysis} confound")
        print('-----------') 
        
        if analysis == "global_signal":
            for FD_threshold in [0.1, 0.5]:                
                
                global_sign_outputs = os.path.join(confound_correction_outputs_base, f"Global_signal_{FD_threshold}")
                confound_correction_outputs=global_sign_outputs
                
                analysis_outputs=os.path.join(analysis_outputs_base, f"SD_{analysis}{FD_threshold}")
                #if not os.path.exists(analysis_outputs):os.makedirs(analysis_outputs)  

                qsub_analysis_rabies(subj_num, specie, template_dir, BIDS_input, preprocess_outputs, confound_correction_outputs, analysis_outputs, Cmd_B_rat_template_path, seed_mask_list)
                print(f"RABIES analysis using: {analysis}_{FD_threshold} confounds")
                print('-----------') 

Specie: **<span style="color:#FFA07A">M</span>**   
Rodent num: <span style="color:#FF5733">0100100</span>  
Analysis: <span style="color:#FF91A4">WM_signal, CSF_signal, global_signal</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1</span>  


Job running: 51041495.dccn-l029.dccn.nl
RABIES analysis using: WM_signal confound
-----------
Job running: 51041496.dccn-l029.dccn.nl
RABIES analysis using: CSF_signal confound
-----------
Job running: 51041497.dccn-l029.dccn.nl
RABIES analysis using: global_signal confound
-----------
Job running: 51041498.dccn-l029.dccn.nl
RABIES analysis using: global_signal_0.1 confounds
-----------
Job running: 51041499.dccn-l029.dccn.nl
RABIES analysis using: global_signal_0.5 confounds
-----------


#### TRASH  
... with previous scripts

In [6]:
# --- Function: launch RABIES analysis, as a job on HPC ---

def qsub_analysis_rabies(subj_num, specie, template_dir, BIDS_input, preprocess_outputs, confound_correction_outputs, Cmd_B_rat_template_path, seed_mask_list):
    
    singularity_path='/opt/singularity/3.10.3/bin/singularity'    
    if not os.path.exists(analysis_outputs):os.makedirs(analysis_outputs)      # Create confound_correction_outputs directory

    # -- Create the full command string --
    analysis_cmd = f"{singularity_path} run " \
                    f"{Cmd_B_rat_template_path} " \
                    f"-B {BIDS_input}:/BIDS_input:ro " \
                    f"-B {preprocess_outputs}:/preprocess_outputs/ " \
                    f"-B {confound_correction_outputs}:/confound_correction_outputs/ " \
                    f"-B {analysis_outputs}:/analysis_outputs/ " \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc analysis /confound_correction_outputs/ /analysis_outputs/ " \
                        f"--seed_list {' '.join(seed_mask_list)}" \
                                                     
#f"--scan_QC_thresholds '{Dice:[0.3]}' " \  
                                                     
    # Submit the job
    cmd = f'echo "Job running: $(echo \'{analysis_cmd}\'| qsub -N Awk:{specie},{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=48:00:00)"'
    subprocess.call(cmd, shell=True)

In [5]:
    # -- Launch: Analysis --    

    class AnalysisConfig(Enum):
        RunAnalysis='0'
        AlreadyAnalysed='1'
     
    rabies_analysis_config_value=str(df.iloc[index]['analysis_config'])    
    rabies_analysis_config = AnalysisConfig(rabies_analysis_config_value)
    
    # -- Print scan infos -- 
    scan_infos=(f'Specie: **<span style="color:#FFA07A">{specie}</span>**   \n'
                f'Rodent num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Preprocess config: <span style="color:#FF91A4">{rabies_analysis_config_value}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n')

    # -- Run RABIES analysis, as a job on the HPC --
    
    if rabies_analysis_config == AnalysisConfig.RunAnalysis:          
         display(Markdown(scan_infos))   
         qsub_analysis_rabies(subj_num, specie, template_dir, BIDS_input, preprocess_outputs, confound_correction_outputs, Cmd_B_rat_template_path, seed_mask_list)
         print("RABIES: analysis")
        
    else : 
        print("RABIES: Already analysed") 

RABIES: Already analysed


In [ ]:
# -- Potentially implement to analysis --
 
    if specie == 'Rat' :
        ICA_prior_map="/project/4180000.36/AwakeRodent/scratch/template/canica_resting_state.nii.gz"  #from https://github.com/grandjeanlab/MultiRat/blob/master/assets/nifti/canica_resting_state.nii.gz
        prior_bold_idx="0 1 3 5 7 8 9 10 11 15 17 18"      # for rats only ??? because different from defaults
        prior_confound_idx="2 4 6 12 13 14 16 19"
        prior_seed_mask_list=
        seed_mask_list=
    else #mouse
        ICA_prior_map=""
        prior_bold_idx=""
        prior_confound_idx=""
        prior_seed_mask_list=""
        seed_mask_list=""
        
# f"--prior_maps {ICA_prior_map} " \
# f"--prior_bold_idx {prior_bold_idx} " \
# f"--prior_confound_idx {prior_confound_idx} " \
# "--data_diagnosis rue" \
# "--DR_ICA True " \
# "--FC_matrix True " \ 
# f"--seed_prior_list {prior_seed_mask_list}"
# "--group_ica='apply'='true','dim'='10','random_seed'='1' " \                                             

